In [1]:
import pymysql
import pandas as pd
from tqdm import tqdm
from openpyxl import load_workbook


## Function to connect to MySQL server

In [2]:
# Set up the database connection parameters
db_params = {
    'host': '127.0.0.1',
    'port': 3308,
    'user': 'ksingh',
    'password': 'c-8t!2H9StisTE0ethlF',
    'database': 'ethermail_bi'
}

# Establish the database connection
conn = pymysql.connect(**db_params)

# Create a cursor object
cursor = conn.cursor()

In [3]:
def connect_mysql(sql_query):
    cursor.execute(sql_query)
    df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
    return df


In [4]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

def save_dataframe_to_gsheet(df, sheet_name):
    json_credentials_path = r'C:\Users\user2\Documents\GitHub\Ethernail_analysis\credentials.json'
    # Set up credentials
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
            "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

    creds = ServiceAccountCredentials.from_json_keyfile_name(json_credentials_path, scope)
    client = gspread.authorize(creds)

    # Open the spreadsheet
    spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/10czZ5f95nYfdXeYqXgFV6r3gTJWxHwNfd1Yiv0w_08U/edit?usp=sharing")

    # Check if the sheet exists
    try:
        worksheet = spreadsheet.worksheet(sheet_name)
        # Clear the existing values
        worksheet.clear()
        DEFAULT_ROWS = 1000
        DEFAULT_COLS = 26
        worksheet.resize(rows=DEFAULT_ROWS, cols=DEFAULT_COLS)
    except gspread.exceptions.WorksheetNotFound:
        # Create a new sheet if it doesn't exist
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows="1000", cols="50")


    # Replace non-standard floating-point values with "NaN" string (or None or any placeholder you prefer)
    df = df.replace([float('inf'), float('-inf')], 'NaN')
    df.fillna('NaN', inplace=True)

    # Convert the entire DataFrame to a list of lists, including headers
    all_values = [df.columns.tolist()] + df.values.tolist()
    
    # Resize the worksheet and update the entire worksheet in one go
    worksheet.update('A1', all_values)

In [5]:
aff_list = pd.read_csv(r'C:\Users\user2\Downloads\Affiliate Program EtherMail_last  - Affiliates List - New.csv')

In [6]:
aff_list

,source,email,afid,social media,date added,email sent,replied,pandadoc,wallet,comment/ beginning date,Read to Earn CTA email
0,Twitter KOL,mualliminbirii@gmail.com,62f7ffc84c1eb40f246d1ad6,https://twitter.com/Muallimin_biri,7.12.2023,13/12/2023,Yes,Yes,0xeA24918a807399a819b74E30baa05f0375Ecdbbd,"reminder sent 20.12.2023, affiliate since 20.1...",NaN
1,Twitter KOL,closepa0920@gmail.com,NaN,https://twitter.com/Goemon_CLOSEPA,7.12.2023,13/12/2023,Yes,Pending,0xbb6FEe4D8B8A121A56AdBBaB64f12792aAA34cD8,"19.12.2023 sent email to get afid, sign pandad...",NaN
2,Twitter KOL,0x1776f8839eb993cc890d9d05edfea47b60a6ada1@eth...,NaN,https://twitter.com/Crypto__Kelv,7.12.2023,13/12/2023,Yes,Pending,0x1776f8839eb993cc890d9d05edfea47b60a6ada1,"reminder sent 20.12.2023, sent email 15.1 to p...",NaN
3,Twitter KOL,karasb7778@gmail.com,6457a7047b34021b83303915,https://twitter.com/zucl1ck,7.12.2023,no,Yes,Yes,NaN,"already was affiliate from before, pending pay...",NaN
4,Twitter KOL,0xD9359E70F0D21b467D96333b4a92a3E8856b56bB@eth...,NaN,https://twitter.com/Rocco_cryptoo,7.12.2023,14/12/2023,Pending,Pending,NaN,reminder sent 20.12.2023,NaN
...,...,...,...,...,...,...,...,...,...,...,...
78,Twitter KOL,NaN,NaN,NaN,NaN,NaN,Pending,Pending,NaN,NaN,NaN
79,Twitter KOL,NaN,NaN,NaN,NaN,NaN,Pending,Pending,NaN,NaN,NaN
80,Twitter KOL,NaN,NaN,NaN,NaN,NaN,Pending,Pending,NaN,NaN,NaN
81,Twitter KOL,NaN,NaN,NaN,NaN,NaN,Pending,Pending,NaN,NaN,NaN


# Data Cleaning

In [7]:
aff_list = aff_list[['email', 'afid']]

In [8]:
aff_list.dropna(axis=0, inplace=True)

C:\Users\user2\AppData\Local\Temp\ipykernel_19188\3461221257.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aff_list.dropna(axis=0, inplace=True)


In [9]:
aff_list.reset_index(drop=True)

,email,afid
0,mualliminbirii@gmail.com,62f7ffc84c1eb40f246d1ad6
1,karasb7778@gmail.com,6457a7047b34021b83303915
2,lsb108@gmail.com,638dd71ffa097cd64782927f
3,elsecretodefi@gmail.com,64b9104da5d5c6e6e2a828d9
4,zentsudanang@gmail.com,6314ca0d7d5eedb3aaaec960
5,ekanosa14@gmail.com,62fa1b197e150cafb0cf345e
6,visitxdream@gmail.com,64220f03b6c469b08466cba0
7,airdropatm@naver.com,6428534df4c8c1e734a2d205
8,cryptopranjalbora@gmail.com,6305e8e6c72123a1d2ee7e9d
9,quant.oracle777@gmail.com,63f5a6b964f69a1bda048928


In [10]:
aff_list.rename(columns={'email': 'affiliate email/twitter', 'afid': 'affiliate ID'}, inplace=True)

C:\Users\user2\AppData\Local\Temp\ipykernel_19188\2957378427.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aff_list.rename(columns={'email': 'affiliate email/twitter', 'afid': 'affiliate ID'}, inplace=True)


In [11]:
aff_list

,affiliate email/twitter,affiliate ID
0,mualliminbirii@gmail.com,62f7ffc84c1eb40f246d1ad6
3,karasb7778@gmail.com,6457a7047b34021b83303915
11,lsb108@gmail.com,638dd71ffa097cd64782927f
13,elsecretodefi@gmail.com,64b9104da5d5c6e6e2a828d9
15,zentsudanang@gmail.com,6314ca0d7d5eedb3aaaec960
23,ekanosa14@gmail.com,62fa1b197e150cafb0cf345e
25,visitxdream@gmail.com,64220f03b6c469b08466cba0
33,airdropatm@naver.com,6428534df4c8c1e734a2d205
44,cryptopranjalbora@gmail.com,6305e8e6c72123a1d2ee7e9d
45,quant.oracle777@gmail.com,63f5a6b964f69a1bda048928


## Query for affliates users without transaction + flagged accounts removed 
### Users without bots and no transactions
	SELECT 
		DATE_FORMAT(uq.created_at, '%Y-%m') AS time_month,
		COUNT(DISTINCT(uq.user_id)) AS count_users_trans
	FROM 
		ethermail_bi.user_query uq
	LEFT JOIN 
		ethermail_bi.flagged_accounts fa 
	ON 
		fa.user_id = uq.user_id 
	WHERE 
		uq.channel_user_id = '6457a7047b34021b83303915'
	AND 
		uq.onboarding_complete = 1
	AND 
		(fa.user_id IS NULL
		OR
		uq.has_transactions = 1)
	GROUP BY 
		DATE_FORMAT(uq.created_at, '%Y-%m');

In [12]:
#df_transaction = connect_mysql(f"SELECT DATE_FORMAT(uq.created_at, '%Y-%m') AS time_month, COUNT(DISTINCT(uq.user_id)) AS count_users_trans FROM ethermail_bi.user_query uq LEFT JOIN ethermail_bi.flagged_accounts fa ON fa.user_id = uq.user_id WHERE uq.channel_user_id = '{affliate_name}' AND uq.onboarding_complete = 1 AND (fa.user_id IS NULL OR uq.has_transactions = 1) GROUP BY DATE_FORMAT(uq.created_at, '%Y-%m');")

## Query for affliates users with transaction  + flagged accounts removed 
### Users without bots and has transactions
	SELECT 
		DATE_FORMAT(uq.created_at, '%Y-%m') AS time_month, -- Time in Months
		COUNT(DISTINCT(uq.user_id)) AS count_users_trans -- User count 
	FROM 
		ethermail_bi.user_query uq
	LEFT JOIN 
		ethermail_bi.flagged_accounts fa 
	ON 
		fa.user_id = uq.user_id 
	WHERE 
		uq.channel_user_id = '6305e8e6c72123a1d2ee7e9d' -- Sample ID
	AND 
		uq.onboarding_complete = 1 -- Onboarded users only
	AND 
		(fa.user_id IS NULL -- Either bots
		OR
		uq.has_transactions = 1) -- Or have completed transaction
	GROUP BY 
		DATE_FORMAT(uq.created_at, '%Y-%m'); -- Grouping for months

In [13]:
#df_no_transaction = connect_mysql(f"SELECT DATE_FORMAT(uq.created_at, '%Y-%m') AS time_month, COUNT(DISTINCT(uq.user_id)) AS count_users_no_trans FROM ethermail_bi.user_query uq LEFT JOIN ethermail_bi.flagged_accounts fa ON fa.user_id = uq.user_id WHERE uq.channel_user_id = '{affliate_name}' AND uq.onboarding_complete = 1 AND fa.user_id IS NULL AND uq.has_transactions = 0 GROUP BY DATE_FORMAT(uq.created_at, '%Y-%m');")

In [14]:
# Remove all spaces from the 'affiliate ID' column
aff_list['affiliate ID'] = aff_list['affiliate ID'].str.replace(' ', '', regex=False)
aff_list = aff_list[['affiliate email/twitter','affiliate ID']]

C:\Users\user2\AppData\Local\Temp\ipykernel_19188\3042826727.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aff_list['affiliate ID'] = aff_list['affiliate ID'].str.replace(' ', '', regex=False)


In [15]:
aff_list

,affiliate email/twitter,affiliate ID
0,mualliminbirii@gmail.com,62f7ffc84c1eb40f246d1ad6
3,karasb7778@gmail.com,6457a7047b34021b83303915
11,lsb108@gmail.com,638dd71ffa097cd64782927f
13,elsecretodefi@gmail.com,64b9104da5d5c6e6e2a828d9
15,zentsudanang@gmail.com,6314ca0d7d5eedb3aaaec960
23,ekanosa14@gmail.com,62fa1b197e150cafb0cf345e
25,visitxdream@gmail.com,64220f03b6c469b08466cba0
33,airdropatm@naver.com,6428534df4c8c1e734a2d205
44,cryptopranjalbora@gmail.com,6305e8e6c72123a1d2ee7e9d
45,quant.oracle777@gmail.com,63f5a6b964f69a1bda048928


# Loop to query each affiliate ID

In [16]:
#Create the first sheet for values
aff_list.to_excel(r"C:\Users\user2\Documents\GitHub\Ethermail_affiliate\affiliate_payout_5Feb2024.xlsx", sheet_name = "Sheet0" ,index=False)

save_dataframe_to_gsheet(aff_list, "Sheet1")

# Specify a writer
with pd.ExcelWriter('affiliate_payout_5Feb2024.xlsx', engine='openpyxl', mode='a') as writer:
    for index, row in tqdm(aff_list.iterrows(), total=len(aff_list), desc="Processing rows"):
        affiliate_name = row['affiliate ID']

        # Dataframe for each affiliate with transaction and no bots
        df_transaction = connect_mysql(f"SELECT DATE_FORMAT(uq.created_at, '%Y-%m') AS time_month, COUNT(DISTINCT(uq.user_id)) AS count_users_trans FROM ethermail_bi.user_query uq WHERE uq.channel_user_id = '{affiliate_name}' AND uq.onboarding_complete = 1 AND uq.invite_level = 1 AND uq.has_transactions = 1 GROUP BY DATE_FORMAT(uq.created_at, '%Y-%m');")

        # Dataframe for each affiliate with no transaction and no bots
        df_no_transaction = connect_mysql(f"SELECT DATE_FORMAT(uq.created_at, '%Y-%m') AS time_month, COUNT(DISTINCT(uq.user_id)) AS count_users_no_trans FROM ethermail_bi.user_query uq LEFT JOIN ethermail_bi.flagged_accounts fa ON fa.user_id = uq.user_id WHERE uq.channel_user_id = '{affiliate_name}' AND uq.onboarding_complete = 1 AND uq.invite_level = 1 AND fa.user_id IS NULL AND uq.has_transactions = 0 GROUP BY DATE_FORMAT(uq.created_at, '%Y-%m');")

        # Merge the two tables together
        df_merge = pd.merge(df_transaction, df_no_transaction, on='time_month', how='left')

        # Create a sheet name, ensuring it is valid and unique
        sheet_name = str(row["affiliate email/twitter"])[:31]  # Max length = 31 chars
        sheet_name = ''.join(x if x.isalnum() else "_" for x in sheet_name)  # Keep alnum chars only


        # Check and adjust if sheet name already exists
        try:
            writer.book[sheet_name]
            suffix = 1
            new_sheet_name = f"{sheet_name[:28]}_{suffix}"
            while new_sheet_name in writer.book.sheetnames:
                suffix += 1
                new_sheet_name = f"{sheet_name[:28]}_{suffix}"
            sheet_name = new_sheet_name
        except KeyError:
            pass  # Sheet name is available, proceed with writing

        # Write DataFrame to a specific sheet
        df_merge.to_excel(writer, sheet_name=sheet_name, index=False)

        #save_dataframe_to_gsheet(df_merge, sheet_name)


C:\Users\user2\AppData\Local\Temp\ipykernel_19188\2062273604.py:4: DeprecationWarning: [Deprecated][in version 6.0.0]: client_factory will be replaced by gspread.http_client types
  save_dataframe_to_gsheet(aff_list, "Sheet1")


C:\Users\user2\AppData\Local\Temp\ipykernel_19188\2657220734.py:38: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update('A1', all_values)
Processing rows: 100%|██████████| 15/15 [10:42<00:00, 42.83s/it]


## Test data for certain ID for validation

In [17]:
affliate_name_1 = "62fb694c7e150cafb0d033d7"

# Datframe for each affliate with transaction and no bots
df_transaction = connect_mysql(f"SELECT DATE_FORMAT(uq.created_at, '%Y-%m') AS time_month, COUNT(DISTINCT(uq.user_id)) AS count_users_trans FROM ethermail_bi.user_query uq WHERE uq.channel_user_id = '{affliate_name_1}' AND uq.onboarding_complete = 1 AND uq.has_transactions = 1 GROUP BY DATE_FORMAT(uq.created_at, '%Y-%m');")

# Datframe for each affliate with no transaction and no bots
df_no_transaction = connect_mysql(f"SELECT DATE_FORMAT(uq.created_at, '%Y-%m') AS time_month, COUNT(DISTINCT(uq.user_id)) AS count_users_no_trans FROM ethermail_bi.user_query uq LEFT JOIN ethermail_bi.flagged_accounts fa ON fa.user_id = uq.user_id WHERE uq.invited_by = '{affliate_name_1}' AND uq.onboarding_complete = 1 AND fa.user_id IS NULL AND uq.has_transactions = 0 GROUP BY DATE_FORMAT(uq.created_at, '%Y-%m');")

# Merge the two tables together
df_merge = pd.merge(df_transaction, df_no_transaction, on='time_month', how='left')
df_merge


,time_month,count_users_trans,count_users_no_trans
0,2022-09,166,4.0
1,2022-10,513,2.0
2,2022-11,1868,3.0
3,2022-12,141,NaN
4,2023-01,209,4.0
5,2023-02,101,NaN
6,2023-03,1626,128.0
7,2023-04,265,9.0
8,2023-05,40,2.0
9,2023-06,22,1.0


In [18]:
aff_list

,affiliate email/twitter,affiliate ID
0,mualliminbirii@gmail.com,62f7ffc84c1eb40f246d1ad6
3,karasb7778@gmail.com,6457a7047b34021b83303915
11,lsb108@gmail.com,638dd71ffa097cd64782927f
13,elsecretodefi@gmail.com,64b9104da5d5c6e6e2a828d9
15,zentsudanang@gmail.com,6314ca0d7d5eedb3aaaec960
23,ekanosa14@gmail.com,62fa1b197e150cafb0cf345e
25,visitxdream@gmail.com,64220f03b6c469b08466cba0
33,airdropatm@naver.com,6428534df4c8c1e734a2d205
44,cryptopranjalbora@gmail.com,6305e8e6c72123a1d2ee7e9d
45,quant.oracle777@gmail.com,63f5a6b964f69a1bda048928


### Check if the afid exists in the channels table
    SELECT 
        c.user_id,
        c.name,
        c.channel_type,
        CASE 
            WHEN c.user_id IS NOT NULL THEN 1
            ELSE 0
        END AS value_exists
    FROM ethermail_bi.channels c
    WHERE c.user_id = '6457a7047b34021b83303915';

In [19]:
df_exists_1 = connect_mysql('''SELECT 
    c.user_id,
    c.name,
    c.channel_type,
    CASE 
        WHEN c.user_id IS NOT NULL THEN 1
        ELSE 0
    END AS value_exists
FROM ethermail_bi.channels c
WHERE c.user_id = '640987f78454790d46c01ddf';''')
df_exists_1

,user_id,name,channel_type,value_exists
0,640987f78454790d46c01ddf,influencers_adsang9999,MARKETING,1


In [20]:
# Initialize an empty dataframe to store all results
final_df = pd.DataFrame(columns= ['user_id', 'name', 'channel_type', 'value_exists'])

for index, row in tqdm(aff_list.iterrows(), total=len(aff_list), desc="Processing rows"):
    affiliate_name = row['affiliate ID']

   # Using a parameterized query to avoid SQL injection
    df_exists = connect_mysql(f"SELECT c.user_id, c.name, c.channel_type, CASE WHEN c.user_id IS NOT NULL THEN 1 ELSE 0 END AS value_exists FROM ethermail_bi.channels c WHERE c.user_id = '{affiliate_name}';")
    
    # Append df_exists to the final dataframe
    final_df = pd.concat([final_df, df_exists], ignore_index=True)



Processing rows: 100%|██████████| 15/15 [00:01<00:00, 11.77it/s]


In [21]:
final_df

,user_id,name,channel_type,value_exists
0,62f7ffc84c1eb40f246d1ad6,influencer_Muallimin_biri,MARKETING,1
1,6457a7047b34021b83303915,influencer_karasb,MARKETING,1
2,638dd71ffa097cd64782927f,influencer_lovestern83,MARKETING,1
3,64b9104da5d5c6e6e2a828d9,influencers_secretodefi,MARKETING,1
4,6314ca0d7d5eedb3aaaec960,influencers_0xZentsu,MARKETING,1
5,62fa1b197e150cafb0cf345e,influencers_thisisnosa,MARKETING,1
6,64220f03b6c469b08466cba0,influencers_connectwithtola,MARKETING,1
7,6428534df4c8c1e734a2d205,influencers_Bounty_ATM,MARKETING,1
8,6305e8e6c72123a1d2ee7e9d,influencer_cryptopranjalbora,MARKETING,1
9,63f5a6b964f69a1bda048928,influencer_quant.oracle777,MARKETING,1


In [22]:
final_df.merge(aff_list, how='outer', left_on='user_id', right_on='affiliate ID')

,user_id,name,channel_type,value_exists,affiliate email/twitter,affiliate ID
0,62f7ffc84c1eb40f246d1ad6,influencer_Muallimin_biri,MARKETING,1,mualliminbirii@gmail.com,62f7ffc84c1eb40f246d1ad6
1,6457a7047b34021b83303915,influencer_karasb,MARKETING,1,karasb7778@gmail.com,6457a7047b34021b83303915
2,638dd71ffa097cd64782927f,influencer_lovestern83,MARKETING,1,lsb108@gmail.com,638dd71ffa097cd64782927f
3,64b9104da5d5c6e6e2a828d9,influencers_secretodefi,MARKETING,1,elsecretodefi@gmail.com,64b9104da5d5c6e6e2a828d9
4,6314ca0d7d5eedb3aaaec960,influencers_0xZentsu,MARKETING,1,zentsudanang@gmail.com,6314ca0d7d5eedb3aaaec960
5,62fa1b197e150cafb0cf345e,influencers_thisisnosa,MARKETING,1,ekanosa14@gmail.com,62fa1b197e150cafb0cf345e
6,64220f03b6c469b08466cba0,influencers_connectwithtola,MARKETING,1,visitxdream@gmail.com,64220f03b6c469b08466cba0
7,6428534df4c8c1e734a2d205,influencers_Bounty_ATM,MARKETING,1,airdropatm@naver.com,6428534df4c8c1e734a2d205
8,6305e8e6c72123a1d2ee7e9d,influencer_cryptopranjalbora,MARKETING,1,cryptopranjalbora@gmail.com,6305e8e6c72123a1d2ee7e9d
9,63f5a6b964f69a1bda048928,influencer_quant.oracle777,MARKETING,1,quant.oracle777@gmail.com,63f5a6b964f69a1bda048928


In [23]:
final_df.to_excel('/Users/Singh/Ethermail/Analysis/Affliate/affiliate_check.xlsx', index=False)

OSError: Cannot save file into a non-existent directory: '\Users\Singh\Ethermail\Analysis\Affliate'